<div style="max-width:1200px"><img src="../_resources/mgnify_banner.png" width="100%"></div>

<img src="../_resources/mgnify_logo.png" width="200px">

[MGnifyR](http://github.com/beadyallen/mgnifyr) is a library that provides a set of tools for easily accessing and processing MGnify data in R, making queries to MGnify databases through the [MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1/). 
The benefits of MGnifyR are that data can either be fetched in tsv format or be directly combined in a phyloseq object to run an analysis in a custom workflow.

This is an interactive code notebook (a Jupyter Notebook). To run this code, click into each cell and press the ▶ button in the top toolbar, or press shift+enter

# Metagenomics bioinformatics at MGnify 2023

## Comparative metagenomics (practical session)

### Normalization methods, alpha & beta diversity, and differentially abundant features

In this notebook we aim to demonstrate how the MGnifyR tool can be used to fetch data and metadata of a MGnify metagenomic analyisis. Then we show diversity metrics calculus and two methods to identify differentially abundant features using taxonomic and functional profiles generated through the [MGnify v5.0 pipeline](https://doi.org/10.1093/nar/gkac1080) for metagenomic assemblies, as shown in the workflow schema below.


<figure>
  <img src="https://www.ebi.ac.uk/metagenomics/static/5e55649e459d5f26ee6c.png" alt="Alt Text" width="800px">
  <figcaption>Mgnify assembly analysis pipeline v5.0</figcaption>
</figure>

## Contents
- [Part 1. Fetching data and preprocessing](#part1)
  - [1.1. Fetch the MGnify Analyses accession](#part1_1)
  - [1.2. Explore and filter samples by metadata](#part1_2)
  - [1.3. Converting into phyloseq object](#part1_3)
- [Part 2. Normalization, alpha diversity indices and taxonomic profiles visualization](#part2)
   - [2.1. Cleaning the OTUs matrix](#part2_1)
   - [2.2. Normalization methods (relative abundance, rarefaction, and CSS)](#part2_2)
   - [2.3. Computing alpha diversity indices](#part2_3)
- [Part 3. Comparative metagenomics at community-level: Beta diversity](#part3)
- [Part 4. Detection of differentially abundant taxa](#part4)
    - [4.1. Association testing](#part4_1)
    - [4.2. Cofounder testing](#part4_2)
- [Part 5. Detection of differentially abundant functions](#part5)
    - [5.1. Fetching KO tables from MGnify and formatting the condition label](#part5_1)
    - [5.2. Generating differentially abundance count tables](#part5_2)
    - [5.3. Selecting pathways to draw](#part5_3)
    - [5.4. Drawing pathways!](#part5_4)
- [References](#refs)

In [ ]:
# Loading libraries:
suppressMessages({    
    library(ALDEx2)
    library(data.table)
    library(dplyr)
    library(ggplot2)
    library(IRdisplay)
    library(KEGGREST)
    library(metagenomeSeq)
    library(microbiomeMarker)
    library(MGnifyR)   
    library(pathview)
    library(phyloseq)
    library(plyr)
    library(SIAMCAT)
    library(stringr)
    library(tidyjson)
    library(tidyverse)
    library(vegan)
})

In [ ]:
# Setting tables and figures size to display (these will be reset later):
options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)
options(repr.plot.width=4, repr.plot.height=4)

In [ ]:
# Setting up functions
collect_pathways = function(ids_list) {
    pathways = list()
    for (id in ids_list) { 
        current_pathway = as.list(keggLink("pathway", id))
        for (index in grep("map", current_pathway)) {        
            clean_id = gsub("*path:", "", current_pathway[index])
            # Discarding chemical structure (map010XX), global (map011XX), and overview (map012XX) maps
            prefix = substring(clean_id, 1, 6)
            if(is.na(match("map010", prefix)) & is.na(match("map011", prefix)) & is.na(match("map012", prefix)) ){
                pathways = append(pathways, clean_id)
            }
        }
    }
    return(pathways)
}

In [ ]:
# Create your session mgnify_client object. For info, "jovyan" is always the user for these Jupyter Docker images. Jovyan as in jovian (a being from the planet Jupiter), but from Jupyter!
mg = mgnify_client(usecache = T, cache_dir = '/home/jovyan/.mgnify_cache')

## Part 1. Fetching data and preprocessing <a id='part1'/>

In this example we are going to fetch MGnify analysis results and metadata for TARA ocean metagenomic study corresponding to size fractions for prokaryotes ([MGYS00002008](https://www.ebi.ac.uk/metagenomics/studies/MGYS00002008#overview)).
Find more information about the [TARA Ocean Project.](https://fondationtaraocean.org/en/expedition/tara-oceans/)

### 1.1. Fetch the MGnify Analyses accession <a id='part1_1'/>

1. The first step is to retrieve the analysis accession list and metadata.

In [ ]:
tara_all = mgnify_analyses_from_studies(mg, 'MGYS00002008')
metadata = mgnify_get_analyses_metadata(mg, tara_all)

<div class="alert alert-block alert-info">
<b>Tip:</b> In case you are interested in running the comparative metagenomic analysis using data from different studies in MGnify, you can adapt the following commands:
</div>
 
    
```R
analyses_accessions = mgnify_analyses_from_studies(mg, c("MGYS1","MGYS2"))

metadata = mgnify_get_analyses_metadata(mg, analyses_accessions)
```

### 1.2. Explore and filter samples by metadata <a id='part1_2'/>

We want to keep only **metagenomic** samples (not amplicon) of 'surface water layer ([ENVO:00002042](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00002042))' and 'mesopelagic zone ([ENVO:00000213](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00000213))' to compare. We also want to filter out results generated with old pipeline versions (<[v5.0](https://www.ebi.ac.uk/metagenomics/pipelines/5.0)). In the following steps we will keep relevant samples and export them to the phyloseq object. Let's first explore the metadata we fetched:

1) Check the number of analyses in the study.

In [ ]:
length(metadata$'analysis_accession')

2) Check the `analysis_experiment-type` to determine whether a filtering is necesary to discard amplicon samples.

In [ ]:
unique(metadata$'analysis_experiment-type')

3) Keep results generated only with the most updated pipeline (v5.0).

In [ ]:
v5_metadata = metadata[which(metadata$'analysis_pipeline-version'=='5.0'), ]
length(v5_metadata$'analysis_accession')

4) Check the `sample_environment-feature` to discover what kind of samples are part of the study and how many of each exists.

<div class="alert alert-block alert-info">
<b>Note:</b> For a comparative study, we need at least 5 samples per group
</div>

In [ ]:
table(v5_metadata$'sample_environment-feature')

Let's keep only samples having [ENVO:00002042](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00002042) or [ENVO:00000213](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00000213) in the `sample_environment-feature` column. We want to create a new dataframe containing the relevant samples.  
We are also going to create a clean label for the environment feature.  
<div class="alert alert-block alert-info">
<b>Note:</b> To speed up the following analysis, we are going to keep only 25 samples per group (by randomly subsampling the accessions)
</div>

5. Filtering out noisy samples and subsampling to 25 samples. We want to create a new dataframe containing the relevant samples. In this step, we will also create a clean label for the environment feature. Additionally, we noticed that some samples have extreme non-sense values in the metadata. We will filter out those samples to reduce the noise on the analysis.

In [ ]:
# Saving the list of Surface water samples in a dataframe
surface_table = v5_metadata[str_detect(v5_metadata$'sample_environment-feature', "surface"), ]

# Reducing the metadata table to keep relevant fields only
variables_to_keep = c('sample_temperature','sample_depth','sample_salinity','sample_chlorophyll sensor','sample_nitrate sensor','sample_oxygen sensor')
reduced_surface_table = surface_table[variables_to_keep]

# Removing rows with extreme values
reduced_surface_table[reduced_surface_table == "99999"] <- NA
reduced_surface_table[reduced_surface_table == "99999.0"] <- NA
reduced_surface_table[reduced_surface_table == "0"] <- NA

clean_reduced_surface_table=na.omit(reduced_surface_table)

# Subsampling to 25
set.seed(345)
random_clean_reduced_surface_table = clean_reduced_surface_table[sample(nrow(clean_reduced_surface_table), 25), ]
random_clean_reduced_surface_table$'env_label' = c(rep('Surface', times = length(rownames(random_clean_reduced_surface_table))))

In [ ]:
# Saving the list of Mesopelagic zone samples in a dataframe
mesopelagic_table = v5_metadata[str_detect(v5_metadata$'sample_environment-feature', "mesopelagic"), ]

# Reducing the metadata table to keep relevant fields only
reduced_mesopelagic_table = mesopelagic_table[variables_to_keep]

# Removing rows with extreme values
reduced_mesopelagic_table[reduced_mesopelagic_table == "99999"] <- NA
reduced_mesopelagic_table[reduced_mesopelagic_table == "99999.0"] <- NA
reduced_mesopelagic_table[reduced_mesopelagic_table == "0"] <- NA

clean_reduced_mesopelagic_table=na.omit(reduced_mesopelagic_table)

# Subsampling to 25
set.seed(345)
random_clean_reduced_mesopelagic_table = clean_reduced_mesopelagic_table[sample(nrow(clean_reduced_mesopelagic_table), 25), ]
random_clean_reduced_mesopelagic_table$'env_label' = c(rep('Mesopelagic', times = length(rownames(random_clean_reduced_mesopelagic_table))))

In [ ]:
# Integrating the metadata table
clean_metadata = rbind(random_clean_reduced_surface_table,random_clean_reduced_mesopelagic_table)
clean_acc = rownames(clean_metadata)

<div class="alert alert-block alert-warning">
<b>Up to you:</b> You can try to re-run the analysis with a different set of samples to compare for instance 'deep chlorophyll maximum layer' versus 'mesopelagic zone'. How would you change the code to filter in relevant samples? Could you used another column in the metadata table?
</div>

### 1.3. Converting into phyloseq object <a id='part1_3'/>

1. Now that we have a new dataframe with 50 samples from either surface or mesopelagic zone water, we are going to create the phyloseq object.

In [ ]:
ps = mgnify_get_analyses_phyloseq(mg, clean_acc)

2. Keep only relevant columns in metadata table and transform numeric variables from characters to numbers. Add the environment label as well.

In [ ]:
# Keeping relevant metadata in the phyloseq object
variables_to_keep = c('sample_temperature','sample_depth','sample_salinity','sample_chlorophyll.sensor','sample_nitrate.sensor','sample_oxygen.sensor')
df = data.frame(sample_data(ps))[variables_to_keep]

# Transforming character to nummeric variables
df[] = lapply(df, function(x) as.numeric(as.character(x)))
sample_data(ps) = df

# Adding the env label                              
sample_data(ps)$'env_label' = clean_metadata$env_label
              
head(sample_data(ps), 3)

## Part 2. Normalization, alpha diversity indices and taxonomic profiles visualization <a id='part2'/>

### 2.1. Cleaning the OTUs matrix <a id='part2_1'/>

1) Remove samples with extremely low coverage – they aren't informative and interfere with the normalization process. The first step is to detect outliers by plotting some histograms.

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)
hist(log10(sample_sums(ps)), breaks = 50, main = "Sample size distribution", xlab = "Sample size (log10)", ylab = "Frequency", col = "#007c80")

We can see that samples with number of reads $\leq 10 ^ {1.5}$ (i.e. $\lesssim 32$) seem to be outliers. 
   Let's filter out the outliers and plot a new histogram.

In [ ]:
ps_good = subset_samples(ps, sample_sums(ps) > 32)
hist(log10(sample_sums(ps_good)), breaks = 50, main = "Sample size distribution", xlab = "Sample size (log10)", ylab = "Frequency", col = "#007c80")

2) Remove singletons. Singletons are OTUs of size one, meaning that only one read was assigned to that OTU. These very low-abundance OTUs could be biologically real (belonging to the rare biosphere ([1](#reference_1))), or they could be false positives due to sequencing artefacts. Singletons observed in only one sample are more likely to be artefacts, and it is good practice to remove them from the OTUs counts table to avoid artificially over-estimating the OTUs richness. You can find more discussion about this in [Robert Edgar's blog](https://drive5.com/usearch/manual/singletons.html).

<div class="alert alert-block alert-warning">
<b>Question:</b> Is the low abundant portion of the microbial community (rare biosphere) biologically relevant? What would be a good approach to study this particular fraction of the microbial communities?
</div>

In [ ]:
ps_final = filter_taxa(ps_good, function(x) sum(x) > 1, prune=TRUE)


<div class="alert alert-block alert-warning">
<b>Up to you:</b> You can try to discard also doubletons or OTUs in size<=3. Is there any difference in the OTUs table?
</div>

3) Show some stats on the sequencing depth across samples.

In [ ]:
max_ratio = max(sample_sums(ps_final))/min(sample_sums(ps_final))

sprintf("The difference in sequencing depth across samples is at most %.2f times larger", max_ratio)

options(repr.plot.width=4, repr.plot.height=5)

boxplot(sample_sums(ps_final), main="Sequencing depth across samples", xlab="", ylab="Number of reads", col="#a6093d")
text(y=boxplot.stats(sample_sums(ps_final))$stats, labels=boxplot.stats(sample_sums(ps_final))$stats, x=1.25)

An approximately 10-fold difference in the library sizes means that we will need to apply a normalization method before continuing with the analysis. The most common normalization methods used in microbiome count data are proportions and rarefaction. However, other methods originally developed to normalize RNA-seq counts have been adapted to differential-abundance analysis in microbiome data.  

A discussion about how to choose the right normalization method is out of the scope of this material, but the topic has been covered in multiple forums and scientific publications. Depending on the downstream analysis we intend to perform, different methods might be appropriate. For instance, to compare groups of samples at community-level through beta-diversity, "...proportions and rarefying produced more accurate comparisons among communities and are the only methods that fully normalized read depths across samples. Additionally, upper quartile, cumulative sum scaling (CSS), edgeR-TMM, and DESeq-VS often masked differences among communities when common OTUs differed, and they produced false positives when rare OTUs differed" ([2](#reference_2)). On the other hand, for detection of differentially abundant species, "both proportions and rarefied counts result in a high rate of false positives in tests for species that are differentially abundant across sample classes" ([3](#reference_3)).    

In the following examples we will show three popular ways of normalization: relative abundance, rarefaction and cummulative sum scaling (CSS). We will produce plots for the taxonomic profile visualization agglomerating the taxonomc labels at Class rank and keep only the most abundant classes to avoid displaying too many unreadable categories in the plot (threshold=1%, i.e. 0.01).

### 2.2. Normalization methods (relative abundance, rarefaction, and CSS) <a id='part2_2'/>

1. **Relative abundance.** The simplest way to normalize the differences in sample size is to transform the OTU counts table into relative abundance through dividing by the number of total reads of each sample. This type of normalization is also referred to as proportions. 

In [ ]:
# Transform taxonomy raw-counts matrix into relative abundance
relab_ps = transform_sample_counts(ps_final, function(x) x/sum(x))

# Agglomerating to class level and store the most abundant classes
psglom = tax_glom(relab_ps, "Class")
top_tss_ps = filter_taxa(psglom, function(x) mean(x) > 0.01, TRUE)

2. **Rarefaction.** Normalization by subsampling is an alternative to relative abundance normalization to obtain an adjusted OTUs count matrix. The method is based on a process of subsampling to the smallest library size in the data set. The algorithm randomly removes reads until the samples reach the same library size. Despite the apparent disadvantage of discarding information from the larger samples, rarefaction is quite popular in microbial ecology. The first step is to find the smallest sample size. We can use the number of observed OTUs in the matrix to do so.

In [ ]:
# Find the smallest sample size
df = as.data.frame(sample_sums(ps_final))
smallest = head(df[order( df[,1] ),],1)

# Rarefying to the smallest sample
ps_rare = rarefy_even_depth(ps_final, sample.size = smallest, replace = FALSE, rngseed = 123, verbose = FALSE)

# Aglomerate taxonomy at Class rank keeping the top 15 classes only
psglom = tax_glom(ps_rare, "Class")
top15 = names(sort(taxa_sums(psglom), decreasing = TRUE)[1:15])
top15_rare_ps = prune_taxa(top15, psglom)

3. **Normalization by cumulative sum scaling (CSS).** For this method we will use the implementation on the `microbiomeMarker` library. Cumulative sum scaling normalization calculates scaling factors as the cumulative sum of gene (or taxa) abundances up to a data-derived threshold. This method is based on the assumption that the count distributions in each sample are equivalent for low abundance genes up to a certain threshold. Only the segment of each sample's count distribution that is relatively invariant across samples is scaled by CSS.

In [ ]:
# Normalizing the OTU counts in the `ps_final` object
ps_CSS = normalize(ps_final, method = "CSS")

# Aglomerate taxonomy at Class rank and keep the top 15 classes only
psglom = tax_glom(ps_CSS, "Class")
top15 = names(sort(taxa_sums(psglom), decreasing = TRUE)[1:15])
top15_css_ps = prune_taxa(top15, psglom)

4. **Community profile visualization.** Visualise the taxonomic profiles using the different normalization methods in barplots at Class rank in two modes.

In [ ]:
options(repr.plot.width=10, repr.plot.height=4)

### Proportions
plot_bar(top_tss_ps, fill = "Class", title="Proportions") + 
    theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), panel.background=element_rect(fill=NA), panel.grid.major=element_line(colour="#ebebeb")) + 
    labs(x='')

### Rarefaction
plot_bar(top15_rare_ps, fill="Class", title="Rarefaction") + 
    theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), panel.background=element_rect(fill=NA), panel.grid.major=element_line(colour="#ebebeb")) + 
    labs(x=NULL)

### Cumulative sum scaling
plot_bar(top15_css_ps, fill="Class", title="Cumulative sum scaling") + 
    theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), panel.background=element_rect(fill=NA), panel.grid.major=element_line(colour="#ebebeb")) + 
    labs(x=NULL)


Stacked barplots are useful to see an overview of the taxonomic profiles across samples and spot samples with a very different profile. However, to compare the abundance profile of particular classes, a single barplot visualization could be more convenient.

In [ ]:
options(repr.plot.width=12, repr.plot.height=2)

### Proportions
plot_bar(subset_taxa(top_tss_ps, Class=='Alphaproteobacteria' | Class=='Dinophyceae' | Class=='Thermoplasmata' ), fill = "Class", title="Proportions") + 
    facet_wrap(~Class) + 
    scale_fill_manual(values=c("#18974c","#d41645","#f49e17")) +
    theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), panel.background=element_rect(fill=NA), panel.grid.major=element_line(colour="#ebebeb")) + 
    labs(x=NULL)

### Rarefaction
plot_bar(subset_taxa(top15_rare_ps, Class=='Alphaproteobacteria' | Class=='Dinophyceae' | Class=='Thermoplasmata' ), fill="Class", title="Rarefaction") + 
    facet_wrap(~Class) + 
    scale_fill_manual(values=c("#18974c","#d41645","#f49e17")) +
    theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), panel.background=element_rect(fill=NA), panel.grid.major=element_line(colour="#ebebeb")) + 
    labs(x=NULL)

### Cumulative sum scaling
plot_bar(subset_taxa(top15_css_ps, Class=='Alphaproteobacteria' | Class=='Dinophyceae' | Class=='Thermoplasmata' ), fill="Class", title="Cumulative sum scaling") + 
    facet_wrap(~Class) + 
    scale_fill_manual(values=c("#18974c","#d41645","#f49e17")) +
    theme(axis.text.x=element_blank(), axis.ticks.x=element_blank(), panel.background=element_rect(fill=NA), panel.grid.major=element_line(colour="#ebebeb")) + 
    labs(x=NULL)

### 2.3. Computing alpha diversity indices <a id='part2_5'/>

Alpha diversity is a measure of species diversity in a particular area or an ecosystem. In terms of microbial ecology, analyzing the sample diversity of sequencing data is commonly the first step in assessing differences between microbial environments. Some of the most popular alpha diversity indices reported in microbial community analyses are Chao1, abundance-based coverage estimators (ACE), Simpson, and Shannon-Weaver.  
Chao1 and ACE are nonparametric estimators that describe diversity as the max number of expected species in the sample (species richness). They consider the proportion of species that have been observed before ("recaptured") to those that are observed only once. Both Chao1 and ACE underestimate true richness at low sample sizes.  
On the other hand, Simpson and Shannon-Weaver, consider relative abundances and depends on both, species richness and the evenness (or equitableness), with which individuals are distributed among the different species. However, both metrics have specific biases. The Shannon-Weaver index places a greater weight on species richness, whereas the Simpson index considers evenness more than richness in its measurement.  
More discussion and examples illustrating how sample size and normalization methods can affect alpha diversity metrics in references [4](#reference_4) and [5](#reference_5).  
According to Pereira *et al.,* (2018)([7](#reference_7)), the best normalization method for metagenomic gene abundance (tested in TARA ocean samples) is CSS for large group sizes. For this reason, we will use this method to show alpha and beta-diversity analyses. 

In [ ]:
options(repr.plot.width=18, repr.plot.height=6)

plot_richness(ps_CSS, x="env_label", color="env_label", title="Cumulative sum scaling", measures=c("Observed", "Chao1", "ACE", "Shannon", "Simpson")) + 
    geom_boxplot() + 
    theme_bw() + 
    scale_color_manual(values=c("#0a5032", "#a1be1f")) + 
    labs(x='', color="Environment")


<div class="alert alert-block alert-warning">
<b>Question:</b> Which group is more diverse?
</div>

## Part 3. Comparative metagenomics at community-level: Beta diversity <a id='part3'/>

Beta (β)-diversity is used in ecology to assess the heterogeneity of local communities. It is defined as the ratio between regional and local species diversity. In simpler terms, it calculates the number of species that are not the same in two different environments. 

Some of the most popular beta diversity measures in microbiome research include Bray-Curtis index (for compositional data), Jaccard index (for presence/absence data, ignoring abundance information), Aitchison distance (Euclidean distance for transformed abundances, aiming to avoid the compositionality bias), and the Unifrac distances (that use the phylogenetic tree information). Only some of the commonly used beta diversity measures are actual distances, therefore, the term dissimilarity or beta-diversity is commonly used. The workflow to assess β-diversity between groups of samples starts with calculating the diversity indices for each pair of samples. This results in a distance (or dissimilarity) matrix that is often used for ordination (e.g., principal coordinates analysis) and data exploration in microbiota data analysis ([6](#reference_6)).

For statistical analysis, [adonis](https://www.rdocumentation.org/packages/vegan/versions/2.6-4/topics/adonis) and [betadisper](https://www.rdocumentation.org/packages/vegan/versions/2.6-4/topics/betadisper) methods are widely used together in β-diversity analyses. 

Adonis tests whether two or more groups have different compositions. Adonis calculates the squared deviations of each site to the centroid and then, performs significance tests using F-tests based on sequential sums of squares from permutations of the raw data. It can be seen as an ANOVA using distance matrices (analogous to MANOVA). A non significant p-value (p>0.05), means that there's no difference in **composition** between groups (null hypothesis of no difference in composition between groups).

Betadisper tests if two or more groups are homogeneously dispersed in relation to their species in studied samples. This test can be done to see if one group has more compositional variance than another. The method uses the distance matrix to calculate the multivariate dispersions (variances; average distance to centroids). Then we use group dispersions to perform an ANOVA test. ANOVA's p-value not significant (p>0.05) means that group **dispersions** are homogenous (null hypothesis of no difference in dispersion between groups).


1) Compute beta diversity using various methods to calculate distance, and perform principle components analysis ploting the first two axes. 

According to Pereira *et al.,* (2018)([7](#reference_7)), the best normalization method for metagenomic gene abundance (tested in TARA ocean samples) is CSS for large group sizes. For this reason, we will use this method to show beta-diversity. Then, using as a guide the steps described in ([8](#reference_8)), we will create a list of suitable distance methods, iterate through them, and display a combined plot. For a better visualization we are going to show the 95% confidence region with an ellipse.

In [ ]:
# Generating the methods list and discarding those that are not included in adonis methods list
dist_methods = unlist(distanceMethodList)
dist_methods = dist_methods[c(-(1:4),-(20:47))]

# Iterating through the list and save the plots
plist = vector("list", length(dist_methods))
names(plist) = dist_methods
for( i in dist_methods ){
    # Calculate distance matrix
    iDist = distance(ps_CSS, method=i)
    # Calculate ordination
    iMDS  = ordinate(ps_CSS, "MDS", distance=iDist)
    ## Make plot
    # Don't carry over previous plot (if error, p will be blank)
    p = NULL
    # Create plot, store as temp variable, p
    p = plot_ordination(ps_CSS, iMDS, color="env_label")
    # Add title to each plot
    p = p + ggtitle(paste("MDS using distance method ", i, sep=""))
    # Save the graphic to file.
    plist[[i]] = p
}

# Create a combined plot
df = ldply(plist, function(x) x$data)
names(df)[1] = "distance"
p = ggplot(df, aes(Axis.1, Axis.2, color=env_label))
p = p + geom_point(size=3, alpha=0.5)
p = p + facet_wrap(~distance, scales="free")
p = p + ggtitle("MDS on various distance metrics for TARA ocean dataset") + 
    stat_ellipse(level=0.95, type="norm", geom="polygon", alpha=0, aes(color=env_label)) + 
    theme_bw() + 
    scale_color_manual(values=c("#0a5032", "#a1be1f")) + 
    labs(color = "Environment") 

options(repr.plot.width=12, repr.plot.height=10)
p

<div class="alert alert-block alert-warning">
<b>Up to you:</b> Remake the plots changing the confidence level used to draw the ellipses. What does it mean?
</div>

<div class="alert alert-block alert-warning">
<b>Question:</b> How do the most commonly used metrics (Bray-Curtis, Jaccard, Euclidian) perform? Which distance metric best segregates the data by water layer?
</div>

2) Separation of the groups observed in some of the ordination plots could be due to composition effect, dispersion effect, or both. To test the composition effect (null hypothesis of no difference in composition between groups) we use a permanova implemented in the `vegan` library's `adonis` function. We will use the distance metric that best segregates the groups and determine whether the two groups of samples have different centroids.

In [ ]:
metadata = data.frame(sample_data(ps_CSS))
css_beta = distance(ps_CSS, method="canberra")
adonis2(css_beta ~ env_label, data = metadata, perm=1e3)

<div class="alert alert-block alert-warning">
<b>Question:</b> What is the conclusion of adonis results?
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> If the p-value is less than some significance level (e.g. 0.05) then we can reject the null hypothesis and conclude that not all group means are equal. Null hypothesis of no difference in composition between groups.
</div>

3) Adonis assumes there is homogeneity of dispersion among groups. Let's test this assumption, to check whether the differences detected by adonis are due to variation in dispersion of the data. The strategy is to run a `betadisper` (also from the `vegan` library) to calculate variances and  evaluate if there's a significant variation between groups through an anova. `Betadisper` is a sister function to `adonis` to study the differences in dispersion within the same geometric framework.

In [ ]:
bd = betadisper(css_beta, metadata$'env_label')
anova(bd)

ANOVA's p-value not significant (p>0.05) means that group dispersions are homogenous (acept the null hypothesis of no difference in dispersion between groups). The general conclusion of `adonis` and `betadisper` is that our groups of samples (Surface and Mesopelagic water layers) are homogeneous on group dispersions (compositions vary similarly within the group) while having significantly different compositions between groups (according to `adonis` p<0.05).

<div class="alert alert-block alert-warning">
<b>Up to you:</b> Try to run the statistical analysis selecting a beta-diversity metric that looks unsuitable for this data (like altGower distance or Raup–Crick dissimilarity). Do you get different conclusions?
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Distance methods available in <code>distance</code> phyloseq function are listed in the object <code>distanceMethodList</code>. You can check such distance methods in detailed in the <code>vegdist</code> <a href="https://search.r-project.org/CRAN/refmans/vegan/html/vegdist.html">documentation</a>
</div>

## Part 4. Detection of differentially abundant taxa <a id='part4'/>

There are many approaches out there to detect differentially abudant taxa (or genes). Here we are using the method implemented in `SIAMCAT` library. `SIAMCAT` is an R package that puts together statistical functions to compare large-scale studies in order to detect microbial community composition changes due to environmental factors. Describing such associations in quantitative models has multiple applications, such as in predicting disease status based on microbiome data. In addition, the pipeline includes the generation of high-quality plots for visual inspection of results ([9](#reference_9)). Noteworthy there are multiple parameters that can be adjusted for a better performance in each step of the analysis. However, in this exercise we are showing the tool behaviour with minimal tuning in default arguments.

### 4.1. Association testing <a id='part4_1'/>

1. Preparing the data. The input for `SIAMCAT` is a matrix with normalized data as relative abundances. We will use the phyloseq object generated in step [2.2](#part2_2) of this notebook and aggregate the data at genus level.

In [ ]:
psglom = tax_glom(relab_ps, "Genus")

2. Creating the `SIAMCAT` object. We first need to create a label to set which group will be used as control for comparisons. We arbitrarily selected Surface as the case group and Mesopelagic as the control group.

In [ ]:
# Creating a siamcat label
sc_label = create.label(meta=sample_data(psglom), label='env_label', case='Surface')

# Creating the siamcat object
siamcat_obj = siamcat(phyloseq=psglom, label=sc_label)


3. Filtering low-abundant features by abundance (threshold=0.001) and prevalence (threshold=0.05).

In [ ]:
siamcat_obj = filter.features(siamcat_obj, filter.method='abundance', cutoff=1e-03)
siamcat_obj = filter.features(siamcat_obj, filter.method='prevalence', cutoff=0.05, feature.type='filtered')

4. Computing differentially abundant taxa on filtered object. The function computes for each species the significance using a non-parametric Wilcoxon test and different effect sizes for the association (e.g. AUC or fold change).

In [ ]:
options(repr.plot.width=12, repr.plot.height=10)
siamcat_obj = check.associations(siamcat_obj)
association.plot(siamcat_obj, panels=c("fc", "prevalence"), prompt = FALSE, verbose = 0)

<div class="alert alert-block alert-info">
<b>Tip:</b> For significantly associated microbial features, the plot shows:
    <ul>
     <li>The abundances of the features across the two different classes (Mesopelagic vs. Surface)</li>
     <li>The significance of the enrichment calculated by a Wilcoxon test (after multiple hypothesis testing correction)</li>
     <li>The generalized fold change of each feature</li>
     <li>The prevalence shift between the two classes</li>
    </ul>
By adding "auroc" to the <code>panels</code> list, you can also visualize the Area Under the Receiver Operating Characteristics Curve (AU-ROC) as non-parametric effect size measure.
</div>

<div class="alert alert-block alert-warning">
<b>Up to you:</b> Play around with the filtering parameters <code>detect.lim</code> and <code>pr.cutoff</code> to see if you can catch more significantly different OTUs. 
</div>

5. Printing the taxonomic labels of differentially abudant OTUs. 

In [ ]:
# The results table of differentially abundant OTUs are stored in associations(siamcat_obj)
diff_otus = as.list(rownames(associations(siamcat_obj)[associations(siamcat_obj)$p.adj < 0.05, ]))

# The taxonomic label per OTU is stored in tax_table(psglom)
tax_table(psglom)[rownames(tax_table(psglom)) %in% diff_otus, ]

### 4.2. Confounder testing <a id='part4_2'/>

1. Test the associated metadata variables for potential confounding influence. As many biological and technical factors beyond the primary phenotype of interest can influence microbiome composition, simple association studies may suffer confounding by other variables, which can lead to spurious results. Associations are visualized either as barplot or Q-Q plot, depending on the type of metadata. Additionally, it evaluates associations among metadata variables using conditional entropy and associations with the label using generalized linear models, producing a correlation heatmap and appropriate quantitative barplots, respectively.

In [ ]:
suppressWarnings(check.confounders(siamcat_obj, fn.plot='conf_check.pdf'))

<div class="alert alert-block alert-info">
<b>Tip:</b> The output file has been created in the current directory. You can open the PDF file in a new tab in this notebook by following this <a href="conf_check.pdf">link</a>.
    You will find the following plots:
    <ul>
      <li>First Plot: Conditional entropies for metadata variables. The conditional entropy check primarily serves to remove nonsensical variables from subsequent checks. Conditional entropy quantifies the unique information contained in one variable (row) respective to another (column). Identical variables and derived variables which share the exact same information will have a value of zero.</li>    
      <li>Second plots: glm regression coefficients + significance + AU-ROCs for the statistical test.</li>
      <li>Third plots (Q-Q plot, boxplot and barplots): Original confounder check descriptive stat plots. The function produces plot for each meta-variable. When the distributions looks very similar for both controls and cases, it is unlikely that such variable would confound the analyses.</li>
      <li>Fourth plots: Variance explained by label versus variance explained by metadata plots. These plots show the variance explained by the label in comparison with the variance explained by the metadata variable for each individual feature. Variables with many features in the upper left corner might be confounding the label associations.</li>
    </ul>
</div>


## Part 5. Detection of differentially abundant functions <a id='part5'/>

In this part of the notebook we will use the subsampled list of Surface and Mesopelagic samples from the TARA Oceans Project, generated on step [1.2.5](#part1_2). After integrating the KO's tables, we run [Aldex2](https://www.bioconductor.org/packages/devel/bioc/vignettes/ALDEx2/inst/doc/ALDEx2_vignette.html) to find differentially abudant genes with KO annotation between Surface and Mesopelagic layers. For this purpose, we will use the `effect` as scale to plot the pathways with the highest number of differentially abundant KOs. Consider that steps involving fetching KO tables and `KEGGREST` queries can take few minutes to run.

To add information to metabolic pathways plots, we use annotation results at gene level by assigning a KEGG Orthology (KO) to putative protein sequences. These results are generated through the [MGnify v5.0 pipeline](https://doi.org/10.1093/nar/gkac1080) for metagenomic assemblies. The [KEGG (Kyoto Encyclopedia of Genes and Genomes) database](https://www.genome.jp/kegg/) is a collection of biological databases, including genetic and metabolic pathways, diseases and drugs, protein-protein interactions, and gene expression. We use the KEGG database to make connections between genes and their biological functions, and define pathways containing the functions to be used as a resource for systems biology.  

The two main key concepts we use in this section of the notebook are:  

1. KEGG Orthologs (KO): Orthologs are genes in different species that evolved from a common ancestral gene and typically retain similar functions. KEGG Orthologs are a set of manually curated genes and proteins grouped into categories based on their functional similarities, particularly their roles in biological pathways and molecular functions. KEGG Orthologues IDs are unique and always starts with the letter 'K' (uppercase) followed by 5 numbers.  

2. KEGG Pathways: Pathways are sets of interconnected biochemical reactions that form a chain leading from an initial reactant to a final product. KEGG pathways provide a high-level overview of the major metabolic pathways in an organism, while KEGG modules provide a more detailed view of the genes and reactions involved in a specific pathway. The IDs of manually drawn reference pathway starts with the word 'map' followed by 5 numbers. Another pathways prefix you will se in the notebook is 'ko' for reference pathway highlighting KO.  

For a better displaying of results, in this notebook we are discarding templates for chemical structure (map010XX), global (map011XX), and overview (map012XX) maps.


### 5.1. Fetching KO tables from MGnify and formatting the condition label <a id='part5_1'/>

1. For this example we need to fetch the KO annotation tables in JSON format from MGnify and reformat it as a dataframe. To speed up this time consuming step, we will subsampled to 10 samples per condition (Surface and Mesopelagic) from the list of accessions in the `ps_final` object generated in section [1.2](#part1_2). Then we integrate the KO annotation tables into a single dataframe. We will discard any sample having zero genes with KO annotation. The KOs table fetching step takes 3 min to run.

In [ ]:
head(sample_data(ps_final), 3)
table(sample_data(ps_final)$env_label)

In [ ]:
# Subsampling to 10 samples per condition
set.seed(123)

surface_sub = data.frame(sample_data(ps)) %>%
  filter(env_label == "Surface") %>%
  sample_n(10, replace = FALSE)

mesopelagic_sub = data.frame(sample_data(ps)) %>%
  filter(env_label == "Mesopelagic") %>%
  sample_n(10, replace = FALSE)

# Storing subsamples in a single dataframe having the MGnify accessions as a column instead of as rownames
sub_table = rbind(surface_sub, mesopelagic_sub)

sub_table$mgnify_accession = rownames(sub_table)
rownames(sub_table) <- NULL

In [ ]:
# Fetching KO tables for 20 samples. This step takes 3 min to run
list_of_dfs = list()
for (accession in sub_table$mgnify_accession) {
    ko_loc = paste0('analyses/',accession,'/kegg-orthologs')
    ko_json = mgnify_retrieve_json(mg, path = ko_loc)
    ko_data = as.data.frame(ko_json %>% spread_all)[ , c("attributes.accession", "attributes.count")]
    colnames(ko_data) = c('ko_id', accession)
    list_of_dfs = append(list_of_dfs, list(ko_data)) 
}

In [ ]:
# Integrating all KO dataframes into a single dataframe
integrated_df = data.frame()
for (df in list_of_dfs){
    integrated_df = merge(integrated_df, df, all = T)
}

## Cleaning the integrated table
# Using the KO id column as row names
row.names(integrated_df) = integrated_df$ko_id
integrated_df$ko_id = NULL

# Converting NA to zero 
integrated_df[is.na(integrated_df)] = 0

# Discarding samples that KOs abundance sum = 0
integrated_df = integrated_df %>% select_if(is.numeric) %>% select_if(~ sum(. != 0) > 0)

In [ ]:
head(integrated_df, c(3,5))

2. Aldex2 requires a list with the condition (Surface or Mesopelagic) sorted according with the features tables (KO counts table). We will use the metadata in the `sub_table` dataframe to map the condition based on the column names of the `integrated_df` object.

In [ ]:
# Mapping the MGYA accession to the condition
sorted_conds = list()
for (sample in colnames(integrated_df)) {
    match = sub_table[sub_table$mgnify_accession %in% sample,]$env_label
    cond = paste(match, collapse = "")
    sorted_conds = append(sorted_conds, cond)    
}
vector_conds = unlist(sorted_conds)
table(vector_conds)

### 5.2. Generating differentially abundance count tables<a id='part5_2'/>

1. Running Aldex2. It takes 2 minutes to complete.

In [ ]:
x.all = aldex(integrated_df, 
              vector_conds, 
              mc.samples=128, 
              test="t", 
              effect=TRUE, 
              include.sample.summary=FALSE, 
              denom="all", 
              verbose=FALSE,
              group = "Mesopelagic"
        )

In [ ]:
head(x.all, 3)

2. The column `effect` in the above output (`x.all` table) contains the log ratio of the sample mean to the reference mean. A positive effect indicates that the sample mean is greater than the reference mean, while a negative effect indicates that the sample mean is lower than the reference mean. In this example, Mesopelagic group is used as reference. We will generate now the matrix of KO-effect.

In [ ]:
ko_matrix = data.matrix(subset(x.all, select = c('effect')))

3. Plotting the effect size (`effect`) and difference (`diff.btw`) to show an overview of differentially abundant functions. Two type of P-value results from the Welch’s t test are shown: In blue marks the expected P-value; in red marks the expected Benjamini-Hochberg corrected P-value. The threshold line indicates P-value = 0.05.

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)

# Effect size plot
par(mfrow=c(1,2))
plot(x.all$effect, 
    x.all$we.ep, 
    log="y", 
    cex=0.7, 
    col=rgb(0,0,1,0.2),  # Blue marks for expected P value of Welch’s t test
    pch=19, 
    xlab="Effect size", 
    ylab="P value", 
    main="Effect size plot")
points(x.all$effect, 
    x.all$we.eBH, 
    cex=0.7, 
    col=rgb(1,0,0,0.2), # Red marks for expected Benjamini-Hochberg corrected P value of Welch’s t test
    pch=19)
abline(h=0.05, lty=2, col="grey")
legend(-3,0.000005, legend=c("P value", "BH-adjusted"), pch=19, col=c("blue", "red"))

# Volcano plot
plot(x.all$diff.btw, 
    x.all$we.ep, 
    log="y", 
    cex=0.7, 
    col=rgb(0,0,1,0.2), # Blue marks for expected P value of Welch’s t test
    pch=19, 
    xlab="Difference", 
    ylab="P value", 
    main="Volcano plot")
points(x.all$diff.btw, 
    x.all$we.eBH, 
    cex=0.7, 
    col=rgb(1,0,0,0.2), # Red marks for expected Benjamini-Hochberg corrected P value of Welch’s t test
    pch=19)
abline(h=0.05, lty=2, col="grey")
legend(-8,0.000005, legend=c("P value", "BH-adjusted"), pch=19, col=c("blue", "red"))

4. Reporting features detected by the Welchs’ or Wilcoxon test individually (blue) or by both (red).

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

found.by.all <- which(x.all$we.eBH < 0.05 & x.all$wi.eBH < 0.05)
found.by.one <- which(x.all$we.eBH < 0.05 | x.all$wi.eBH < 0.05)

plot(x.all$diff.win, x.all$diff.btw, pch=19, cex=1, col=rgb(0,0,0,0.3),
 xlab="Dispersion", ylab="Difference")
points(x.all$diff.win[found.by.one], x.all$diff.btw[found.by.one], pch=19,
 cex=1, col=rgb(0,0,1,0.5))
points(x.all$diff.win[found.by.all], x.all$diff.btw[found.by.all], pch=19,
 cex=1, col=rgb(1,0,0,1))
abline(0,1,lty=2)
abline(0,-1,lty=2)


### 5.3. Selecting pathways to draw<a id='part5_3'/>

1. We will use the intersection of both testing methods (Welchs’ or Wilcoxon) to find the pathways with differentially abundant KOs. This step takes 3 min to run.

In [ ]:
# Storing the list of KOs found as differentially abundant by both methods
kos_list = list()
for (index in found.by.all){
    current_ko = rownames(x.all)[index]
    kos_list = append(kos_list, current_ko)
}

In [ ]:
# Generating the list of pathways with differentially abundant KOs. This step takes 3 min to complete
ko_pathways = collect_pathways(kos_list)

2. Finding the pathways with the highest number of significant KOs.

In [ ]:
# Counting the number of differentially abundant KOs per pathway
pathways_counts = list()
for (path_element in ko_pathways) {
    if (path_element %in% names(pathways_counts)) {
        new_value = pathways_counts[[path_element]] + 1
        pathways_counts[path_element] = new_value       
    } else {
        pathways_counts[path_element] = 1 
    }
}

In [ ]:
# Saving the pathways with the highest number of differentially abundant KOs
top_to_plot = names(tail(pathways_counts[order(unlist(pathways_counts))], 12))
top_to_plot

### 5.4. Drawing pathways!<a id='part5_4'/>

1. For this type of data, values can have both negative and positive fractions, that's why we use both directions when plotting. Pathview will display a long message while working with some warnings. You will see new files with `png` and `xml` extension appearing on the left-hand side panel, we will clean that in the next step. This step takes around 2 minutes to complete.

In [ ]:
for (p in top_to_plot) {
    nude_id =  gsub("map", "", p)
    pathview(gene.data = ko_matrix, 
        species = "ko", 
        pathway.id = nude_id, 
        both.dirs = TRUE, 
        low = c("#bd066b", "#bd066b"),  
        mid = c("#c9c9c9" , "#c9c9c9"), 
        high = c("#02b3ad" , "#02b3ad")
    )
}

2. Cleaning the working directory.

In [ ]:
if (!dir.exists("output_plots")){
    dir.create("output_plots")
    dir.create("output_plots/functions")
}
if (!dir.exists("output_plots/functions")){
    dir.create("output_plots/functions")
}

file.copy(from=list.files(pattern="./*pathview.png"), to="./output_plots/functions/", overwrite = TRUE)

png_files = list.files(path = ".", pattern = "*.png")
xml_files = list.files(path = ".", pattern = "*.xml")
files = c(png_files, xml_files)
unlink(files)

3. The following are two examples of pathways having a high number of differentially abundant KOs. Other maps are stored at the `output_plots/functions/` directory.

In [ ]:
display_png(file='./output_plots/functions/ko00720.pathview.png')

In [ ]:
display_png(file='./output_plots/functions/ko00680.pathview.png')

In [ ]:
# Get pathview help
#?pathview

 ### References: <a id='refs'/></a>

<a id="reference_1">1.</a> Lynch, M., Neufeld, J. Ecology and exploration of the rare biosphere. Nat Rev Microbiol 13:217–229 (2015) [DOI:10.1038/nrmicro3400](https://doi.org/10.1038/nrmicro3400)
    
<a id="reference_2">2.</a> McKnight, DT., Huerlimann, R., Bower, DS. et al. Methods for Normalizing Microbiome Data: An Ecological Perspective. Methods in Ecology and Evolution / British Ecological Society 10(3):389–400 (2019) [DOI:10.1111/2041-210X.13115](https://doi.org/10.1111/2041-210X.13115)

<a id="reference_3">3.</a> McMurdie, PJ., Holmes, S. Waste Not, Want Not: Why Rarefying Microbiome Data Is Inadmissible. PLoS Comput Biol 10(4):e1003531 (2014) [DOI:10.1371/journal.pcbi.1003531](https://doi.org/10.1371/journal.pcbi.1003531)

<a id="reference_4">4.</a> Willis, A. Rarefaction, Alpha Diversity, and Statistics. Front. Microbiol. Sec. Terrestrial Microbiology 10:2407 (2019) [DOI:10.3389/fmicb.2019.02407](https://doi.org/10.3389/fmicb.2019.02407)
    
<a id="reference_5">5.</a> Kim, BR., Shin, J., Guevarra, R. et al. Deciphering Diversity Indices for a Better Understanding of Microbial Communities. J Microbiol Biotechnol 28;27(12):2089-2093 (2017) [DOI:10.4014/jmb.1709.09027](https://doi.org/10.4014/jmb.1709.09027)

<a id="reference_6">6.</a> Wagner, BD., Grunwald, GK., Zerbe, GO. et al. On the Use of Diversity Measures in Longitudinal Sequencing Studies of Microbial Communities. Front. Microbiol. 9:1037 (2018) [DOI:10.3389/fmicb.2018.01037](https://doi.org/10.3389/fmicb.2018.01037)
    
<a id="reference_7">7.</a> Pereira, M., Wallroth, M., Jonsson, V. et al. Comparison of normalization methods for the analysis of metagenomic gene abundance data. BMC Genomics 19,274 (2018) [DOI:10.1186/s12864-018-4637-6](https://doi.org/10.1186/s12864-018-4637-6)
    
<a id="reference_8">8.</a> McMurdie, P., Holmes, S. Tutorial of 2018: The distance function in phyloseq. [Available at website](https://joey711.github.io/phyloseq/distance.html)

<a id="reference_9">9.</a> Wirbel, J., Zych, K., Essex, M. et al. Microbiome meta-analysis and cross-disease comparison enabled by the SIAMCAT machine learning toolbox. Genome Biol 22:93 (2021) [DOI:10.1186/s13059-021-02306-1](https://doi.org/10.1186/s13059-021-02306-1)
        
<a id="reference_10">10.</a> Tibshirani, R. Regression Shrinkage and Selection via the Lasso. Journal of the Royal Statistical Society. Series B (Methodological) 58(1):267-288 (1996) [DOI:10.1111/j.2517-6161.1996.tb02080.x](https://doi.org/10.1111/j.2517-6161.1996.tb02080.x)


#### R libraries:
 - `library(ALDEx2)` Gloor GB, Macklaim JM, Fernandes AD (2016). Displaying Variation in Large Datasets: a Visual Summary of Effect Sizes. Journal of Computational and Graphical Statistics, 2016 http://doi.org/10.1080/10618600.2015.1131161. R package version 1.30.0.  

 - `library(data.table)` Matt Dowle and Arun Srinivasan (2023). data.table: Extension of `data.frame`. R package version 1.14.8. https://CRAN.R-project.org/package=data.table  

 - `library(dplyr)` Hadley Wickham, Romain François, Lionel Henry, Kirill Müller and Davis Vaughan (2023). dplyr: A Grammar of Data Manipulation. R package version 1.1.2. https://CRAN.R-project.org/package=dplyr  

 - `library(ggplot2)` Wickham H (2016). ggplot2: Elegant Graphics for Data Analysis. Springer-Verlag New York. ISBN 978-3-319-24277-4, https://ggplot2.tidyverse.org.  

 - `library(IRdisplay)` Thomas Kluyver, Philipp Angerer and Jan Schulz (NA). IRdisplay: 'Jupyter' Display Machinery. R package version 1.1. https://github.com/IRkernel/IRdisplay  

 - `library(KEGGREST)` Dan Tenenbaum and Bioconductor Package Maintainer (2021). KEGGREST: Client-side REST access to the Kyoto Encyclopedia of Genes and Genomes (KEGG). R package version 1.38.0.  

 - `library(metagenomeSeq)` Paulson JN, Olson ND, Braccia DJ, Wagner J, Talukder H, Pop M, Bravo HC (2013). metagenomeSeq: Statistical analysis for sparse high-throughput sequencing. Bioconductor package, http://www.cbcb.umd.edu/software/metagenomeSeq.  

 - `library(MGnifyR)` Ben Allen (2022). MGnifyR: R interface to EBI MGnify metagenomics resource. R package version 0.1.0.  

 - `library(microbiomeMarker)` Cao Y, Dong Q, Wang D, Zhang P, Liu Y, Niu C (2022). “microbiomeMarker: an R/Bioconductor package for microbiome marker identification and visualization.” Bioinformatics. doi:10.1093/bioinformatics/btac438.  

 - `library(pathview)` Luo, W. and Brouwer C., Pathview: an R/Bioconductor package for pathway-based data integration and visualization. Bioinformatics, 2013, 29(14): 1830-1831, doi: 10.1093/bioinformatics/btt285. R package version 1.38.0.  

 - `library(phyloseq)` McMurdie and Holmes (2013) phyloseq: An R Package for Reproducible Interactive Analysis and Graphics of Microbiome Census Data. PLoS ONE. 8(4):e61217  

 - `library(plyr)` Wickham H (2011). “The Split-Apply-Combine Strategy for Data Analysis.” Journal of Statistical Software, 40(1), 1–29. https://www.jstatsoft.org/v40/i01/.  

 - `library(SIAMCAT)` Wirbel J, Zych K, Essex M, Karcher N, Kartal E, Salazar G, Bork P, Sunagawa S, Zeller G (2021). “Microbiome meta-analysis and cross-disease comparison enabled by the SIAMCAT machine learning toolbox.” Genome Biology. https://doi.org/10.1186/s13059-021-02306-1.  
    
 - `library(stringr)` Wickham H (2022). stringr: Simple, Consistent Wrappers for Common String Operations. https://stringr.tidyverse.org, https://github.com/tidyverse/stringr.  

 - `library(tidyjson)` Jeremy Stanley and Cole Arendt (2023). tidyjson: Tidy Complex 'JSON'. R package version 0.3.2. https://CRAN.R-project.org/package=tidyjson  

 - `library(tidyverse)` Wickham H, Averick M, Bryan J, Chang W, McGowan LD, François R, Grolemund G, Hayes A, Henry L, Hester J, Kuhn M, Pedersen TL, Miller E, Bache SM, Müller K, Ooms J, Robinson D, Seidel DP, Spinu V, Takahashi K, Vaughan D, Wilke C, Woo K, Yutani H (2019). “Welcome to the tidyverse.” Journal of Open Source Software, 4(43), 1686. doi:10.21105/joss.01686.  
    
 - `library(vegan)` Oksanen, J., Blanchet, F. G., Friendly, M., Kindt, R., Legendre, P., McGlinn, D., ... & Wagner, H. (2019). vegan: Community Ecology Package. R package version 2.6-4. Retrieved from https://CRAN.R-project.org/package=vegan.  



#### Going deeper:

Documentation and more MGnifyR code and exercises available [on GitHub](https://beadyallen.github.io/MGnifyR/) and [on rdrr site](https://rdrr.io/github/beadyallen/MGnifyR/f/vignettes/MGnifyR.Rmd). Online tutorial available here:
https://www.ebi.ac.uk/training/online/courses/metagenomics-bioinformatics/mgnifyr/

KEGGREST documentation with multiple examples:
https://www.bioconductor.org/packages/devel/bioc/vignettes/KEGGREST/inst/doc/KEGGREST-vignette.html

Pathview user manual:
https://www.bioconductor.org/packages/release/bioc/vignettes/pathview/inst/doc/pathview.pdf

Phyloseq tutorials available [on GitHub](https://joey711.github.io/phyloseq/index.html)
    
Visit the [SIAMCAT documentation](https://siamcat.embl.de/)